In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import faiss
import duckdb

/Users/felix/MSE/03_projects/MT/06_rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
QUESTIONS = '../data/kilt_fever_train_tiny.jsonl'
MODEL = 'jinaai/jina-embeddings-v3'
INDEX = '../data/kilt_wiki_tiny.index'
SOURCE = '../data/kilt_wiki_tiny.duckdb'

In [3]:
df_q = pd.read_json(QUESTIONS, lines=True)
df_q

,id,input,meta,output
0,150448,Roman Atwood is a content creator.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
1,156709,Adrienne Bailon is an accountant.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'REFUTES', 'meta': {'score': -1}, ..."
2,129629,Homeland is an American television spy thrille...,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
3,33078,The Boston Celtics play their home games at TD...,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
4,6744,The Ten Commandments is an epic film.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
5,226034,Tetris has sold millions of physical copies.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
6,76253,There is a movie called The Hunger Games.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
7,138503,"Stranger Things is set in Bloomington, Indiana.","{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'REFUTES', 'meta': {'score': -1}, ..."
8,129983,Ryan Seacrest is a person.,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'SUPPORTS', 'meta': {'score': -1},..."
9,73170,Puerto Rico is not an unincorporated territory...,"{'left_context': '', 'mention': '', 'right_con...","[{'answer': 'REFUTES', 'meta': {'score': -1}, ..."


In [4]:
model = SentenceTransformer(MODEL, trust_remote_code=True)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: aeeef10b-c734-4f5f-b5e5-059c37421f18)')' thrown while requesting HEAD https://huggingface.co/jinaai/jina-embeddings-v3/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


KeyboardInterrupt: 

In [ ]:
index = faiss.read_index(INDEX)

In [ ]:
index.is_trained

True

In [ ]:
def retriev(item, index, con, k=5):
    task = "retrieval.query"
    query = model.encode(
        item['input'],
        task=task,
        prompt_name=task,
    )

    distances, indices = index.search(np.array([query]), k)

    # TODO: Why???
    indices = indices+1

    result = con.execute("""
        SELECT *
        FROM paragraph
        WHERE global_id IN ({})
        """.format(",".join(map(str, indices[0])))).df()

    result['d'] = distances[0]

    print(item['input'])
    # print(distances) # TODO: Find out why the distance is always the same?
    return result

con = duckdb.connect(SOURCE)
retriev(df_q.iloc[0], index, con)



Roman Atwood is a content creator.


,wikipedia_id,wikipedia_title,global_id,index,text,d
0,43386202,Roman Atwood,4,23,"In March 2014, Roman, along with his accomplic...",0.718306
1,43386202,Roman Atwood,14,22,Section::::Legal issues.\n,0.718306
2,43386202,Roman Atwood,24,21,"In 2016, Atwood and fellow YouTuber Yousef Era...",0.718306
3,43386202,Roman Atwood,34,20,Section::::Career.:Tour.\n,0.718306
4,43386202,Roman Atwood,44,19,"In 2016, Atwood, Vitaly Zdorovetskiy and Denni...",0.718306


In [ ]:
con.execute("SELECT * FROM paragraph WHERE wikipedia_id = 43386202").df()

,wikipedia_id,wikipedia_title,global_id,index,text
0,43386202,Roman Atwood,4,23,"In March 2014, Roman, along with his accomplic..."
1,43386202,Roman Atwood,14,22,Section::::Legal issues.\n
2,43386202,Roman Atwood,24,21,"In 2016, Atwood and fellow YouTuber Yousef Era..."
3,43386202,Roman Atwood,34,20,Section::::Career.:Tour.\n
4,43386202,Roman Atwood,44,19,"In 2016, Atwood, Vitaly Zdorovetskiy and Denni..."
5,43386202,Roman Atwood,54,18,"Section::::Career.:Film.:""Natural Born Prankst..."
6,43386202,Roman Atwood,64,17,Section::::Career.:Film.\n
7,43386202,Roman Atwood,74,16,"A prank titled ""Killing My Own Kid PRANK!!"", p..."
8,43386202,Roman Atwood,84,15,Section::::Career.:YouTube.:Killing My Kid Pra...
9,43386202,Roman Atwood,94,14,"His video ""Anniversary Prank Backfires!!"" was ..."


In [ ]:
con.execute("""
SELECT * FROM paragraph
WHERE global_id IN ({})
""".format(",".join(map(str, [441])))).df()

,wikipedia_id,wikipedia_title,global_id,index,text
0,43376,Boston Celtics,441,143,"BULLET::::- 48. Steve Kuberski (2,929)\n"


In [ ]:
res = con.execute("SELECT * FROM paragraph").df()

res['identifier'] = res.apply(lambda row: f"{row['wikipedia_id']}_{row['index']}", axis=1)


ids = res['identifier']
res[ids.isin(ids[ids.duplicated()])].sort_values("identifier")

,wikipedia_id,wikipedia_title,global_id,index,text,identifier


In [ ]:
res

,wikipedia_id,wikipedia_title,global_id,index,text,identifier
0,76383,The Ten Commandments (1956 film),1,128,"BULLET::::- Costume design drawings for ""The T...",76383_128
1,23041,Puerto Rico,2,335,United Nations (U.N.) Declaration on Puerto Ri...,23041_335
2,30339,Tetris,3,112,"BULLET::::- List of ""Tetris"" variants\n",30339_112
3,43386202,Roman Atwood,4,23,"In March 2014, Roman, along with his accomplic...",43386202_23
4,9233381,Ryan Seacrest,5,57,BULLET::::- DigitalReview.net Biography and re...,9233381_57
...,...,...,...,...,...,...
1166,23041,Puerto Rico,1167,5,Puerto Ricans have been citizens of the United...,23041_5
1167,23041,Puerto Rico,1168,4,Originally populated by the indigenous Taíno p...,23041_4
1168,23041,Puerto Rico,1169,3,"An archipelago among the Greater Antilles, Pue...",23041_3
1169,23041,Puerto Rico,1170,2,"Puerto Rico (), officially the Commonwealth of...",23041_2


In [ ]:

def extract_wikipedia_link(row):

    item = row['output'][0]['provenance'][0]

    return {
        'wikipedia_id': item['wikipedia_id'],
        'section': item['section'],
        'paragraph_id': item['start_paragraph_id'],
        'start_paragraph_id': item['start_paragraph_id'],
        'end_paragraph_id': item['end_paragraph_id']
    }

In [ ]:
row = df_q.iloc[0]

ground_truth = extract_wikipedia_link(row)
retrieved_documents = retriev(row, index, con)



Roman Atwood is a content creator.


In [ ]:
retrieved_documents

,wikipedia_id,wikipedia_title,global_id,index,text,d
0,43386202,Roman Atwood,4,23,"In March 2014, Roman, along with his accomplic...",0.718306
1,43386202,Roman Atwood,14,22,Section::::Legal issues.\n,0.718306
2,43386202,Roman Atwood,24,21,"In 2016, Atwood and fellow YouTuber Yousef Era...",0.718306
3,43386202,Roman Atwood,34,20,Section::::Career.:Tour.\n,0.718306
4,43386202,Roman Atwood,44,19,"In 2016, Atwood, Vitaly Zdorovetskiy and Denni...",0.718306


In [ ]:
ground_truth['wikipedia_id'] in retrieved_documents['wikipedia_id'].to_list()

True

In [ ]:
ground_truth['paragraph_id'] in retrieved_documents['index'].to_list()

False